<a href="https://colab.research.google.com/github/Vinhnv0901/student_project/blob/master/result_recommend.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
import pandas as pd
import os
import zipfile
import numpy as np
import matplotlib.pyplot as plt
import string
from sklearn.model_selection import train_test_split

In [22]:
!git clone "https://github.com/Vinhnv0901/student_project.git"

fatal: destination path 'student_project' already exists and is not an empty directory.


# load data from github

In [23]:
IS = pd.read_excel("/content/student_project/IS.xlsx")
TS = pd.read_excel("/content/student_project/TS.xlsx")
GS = pd.read_excel("/content/student_project/GS.xlsx")
GS_replace = pd.read_excel("/content/student_project/GS_replace.xlsx")
data_train = pd.read_excel("/content/student_project/data_train.xlsx")
data_test = pd.read_excel("/content/student_project/data_test.xlsx")
data_merge = pd.read_excel("/content/student_project/data_merge.xlsx")

In [25]:
IS.index = IS['mssv']
GS.index = GS['mssv']
TS.index = TS['mamh']
GS_replace.index = GS_replace['mssv']

In [26]:
student_test = GS.index

In [27]:
IS = IS[IS['mssv'].isin(student_test)]

#label học kỳ một của năm 2016

In [28]:
data_label_test_1 = data_merge[data_merge['mssv'].isin(student_test) & (data_merge['namhoc'] ==2016 ) & (data_merge['hocky'] == 1)]

In [29]:
label_test_1 = []
for i in range(student_test.shape[0]):
  stu = data_label_test_1[data_label_test_1['mssv'] == student_test[i]]
  if stu.empty:
    label_test_1.append([])
  else:
    label_test_1.append(stu['mamh'].tolist())

#label học kỳ hai của năm 2016

In [30]:
data_label_test_2 = data_merge[data_merge['mssv'].isin(student_test) & (data_merge['namhoc'] ==2016 ) & (data_merge['hocky'] == 2)]

In [31]:
label_test_2 = []
for i in range(student_test.shape[0]):
  stu = data_label_test_2[data_label_test_2['mssv'] == student_test[i]]
  if stu.empty:
    label_test_2.append([])
  else:
    label_test_2.append(stu['mamh'].tolist())

#xóa các cột index để dễ tính toán

In [32]:
IS = IS.drop('mssv', axis = 1)
TS = TS.drop('mamh', axis = 1)
GS = GS.drop('mssv', axis = 1)
GS_replace = GS_replace.drop('mssv', axis = 1)

#dự đoán ra k môn có Score cao nhất
##dự đoán trên Interest score

In [14]:
k = 15
IS_k = IS.apply(lambda row: row.nlargest(k), axis=1)

In [15]:
predict_IS = []
for i in range(IS_k.shape[0]):
  pre = []
  for j in range(IS_k.shape[1]):
    if not pd.isna(IS_k.iloc[i,j]):
      pre.append(IS_k.columns[j])
  predict_IS.append(pre)

In [17]:
predict_IS[2]

['EN004',
 'ENBT',
 'IT001',
 'IT002',
 'IT003',
 'IT004',
 'IT005',
 'IT006',
 'IT009',
 'MA001',
 'MA002',
 'MA003',
 'PE001',
 'PH001',
 'PH002']

In [16]:
recall = []
precision = []
for i in range(student_test.shape[0]):
  lab = label_test_1[i]
  pre = predict_IS[i]
  if pre:
    true_true = set(lab) & set(pre)
    if true_true:
      recall.append(len(true_true)/len(lab))
      precision.append(len(true_true)/len(pre))
    else:
      recall.append(0)
      precision.append(0)
r = sum(recall)/len(recall)
p = sum(precision)/len(precision)
f1 = (2*r*p)/(r+p)
print('recall khi sử dụng Interest Score: ', r)
print('precision khi sử dụng Interest Score: ', p)
print('f1-Score khi sử dụng Interest Score: ', f1)

recall khi sử dụng Interest Score:  0.15075585789871504
precision khi sử dụng Interest Score:  0.06020408163265302
f1-Score khi sử dụng Interest Score:  0.08604589094684806


## dự đoán dựa trên Timing and popularity based Score

In [18]:
k = 15
TS_k = TS.apply(lambda row: row.nlargest(k), axis=0)

In [19]:
predict_TS = []
for i in range(TS_k.shape[1]):
  pre = []
  for j in range(TS_k.shape[0]):
    if not pd.isna(TS_k.iloc[j,i]):
      pre.append(TS_k.index[j])
  predict_TS.append(pre)

In [20]:
recall = []
precision = []
for i in range(student_test.shape[0]):
  stu = data_label_test_1[data_label_test_1['mssv'] == student_test[i]]
  if not stu.empty:
    term = stu.loc[stu.index[0], 'soHocKy']
    lab = label_test_1[i]
    pre = predict_TS[term - 1]
    true_true = set(lab) & set(pre)
    if true_true:
      recall.append(len(true_true)/len(lab))
      precision.append(len(true_true)/len(pre))
    else:
      recall.append(0)
      precision.append(0)
r = sum(recall)/len(recall)
p = sum(precision)/len(precision)
f1 = (2*r*p)/(r+p)
print('recall khi sử dụng Interest Score: ', r)
print('precision khi sử dụng Interest Score: ', p)
print('f1-Score khi sử dụng Interest Score: ', f1)

recall khi sử dụng Interest Score:  0.3903619413216345
precision khi sử dụng Interest Score:  0.15099168266154825
f1-Score khi sử dụng Interest Score:  0.2177556545516473


##dự đoán dựa trên Grade-based Score

In [ ]:
k = 15
GS_k = GS.apply(lambda row: row.nlargest(k), axis=1)

In [ ]:
predict_GS = []
for i in range(GS_k.shape[0]):
  pre = []
  for j in range(GS_k.shape[1]):
    if not pd.isna(GS_k.iloc[i,j]):
      pre.append(GS_k.columns[j])
  predict_GS.append(pre)

In [ ]:
recall = []
precision = []
for i in range(student_test.shape[0]):
  lab = label_test_1[i]
  pre = predict_GS[i]
  if pre:
    true_true = set(lab) & set(pre)
    if true_true:
      recall.append(len(true_true)/len(lab))
      precision.append(len(true_true)/len(pre))
    else:
      recall.append(0)
      precision.append(0)
r = sum(recall)/len(recall)
p = sum(precision)/len(precision)
f1 = (2*r*p)/(r+p)
print('recall khi sử dụng Interest Score: ', r)
print('precision khi sử dụng Interest Score: ', p)
print('f1-Score khi sử dụng Interest Score: ', f1)

recall khi sử dụng Interest Score:  0.04626928723787358
precision khi sử dụng Interest Score:  0.01780104712041883
f1-Score khi sử dụng Interest Score:  0.02571054983867054


In [ ]:
course_grade_train = pd.read_excel("/content/student_project/course_grade_train.xlsx")

In [ ]:
course_grade_train.head(10)

,ADENG1,ADENG2,ADENG3,CE101,CE103,CE104,CE105,CE107,CE114,CE117,...,SMET2,SPCH3723,SPCH3724,SS001,SS002,SS003,SS004,SS005,STAT4033,mssv
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,...,0,0.0,0.0,3.3,0.0,0.0,0.0,0.0,0.0,F451D99AXPvAibaEXe+fPSOag5leTcWuR+r937fj
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,...,0,0.0,0.0,6.7,5.6,0.0,8.5,0.0,0.0,6988D28AXPvAibaEXe9MSDlNDFbYdo3rYOhSopw2
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,...,0,0.0,0.0,4.5,6.5,0.0,0.0,0.0,0.0,7E308531XPvAibaEXe879+AOg1gh8i58Q/VMq7RU
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DD10B95EXPvAibaEXe/PQrWCRBKWOUTASXEgaMpR
4,0.0,0.0,0.0,5.9,6.7,7.3,5.9,8.6,0,8.0,...,0,0.0,0.0,4.5,7.0,8.0,9.0,0.0,0.0,4CD37F46XPvAibaEXe8ge2UHv4J4Byyo6uzLKM42
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CEF4F5F1XPvAibaEXe9ej2kTw4OUmtE+HmhDvEXe
6,7.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,...,0,9.1,10.0,7.0,7.5,6.5,0.0,0.0,9.7,AB4D8F99XPvAibaEXe/YKN5ysyYL7EjErJAJK5l+
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,...,0,0.0,0.0,5.0,5.2,3.0,8.3,0.0,0.0,D349E207XPvAibaEXe/aPLdUCtOgChkEtg7JE0Ca
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,...,0,0.0,0.0,0.0,7.4,6.4,6.2,0.0,0.0,C9CD6314XPvAibaEXe8LX4hhYKrFZr9FZE839SqA
9,5.0,7.1,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,...,0,6.7,8.0,6.0,7.0,5.2,0.0,0.0,7.0,D78141E5XPvAibaEXe9j81k8MH0oJ4zch8KMTSfQ


In [ ]:
data_train[(data_train['mamh'] == course_grade_train.columns[4]) & (data_train['mssv'] == course_grade_train.loc[course_grade_train.index[4], 'mssv'])]

,level_0,index,mssv,mamh,malop,sotc,namhoc,hocky,diem,trangthai,...,noisinh,lopsh,khoa,hedt,khoahoc,chuyennganh2,tinhtrang,KhoaMH,namNhapHoc,soHocKy
14758,18902,21777,4CD37F46XPvAibaEXe8ge2UHv4J4Byyo6uzLKM42,CE103,CE103.G12,4,2015,1,6.7,1,...,Thanh Hoá,KTMT0001,KTMT,CQUI,8,D520214,3,KTMT,2013,5


In [ ]:
course_grade_train.columns[4], course_grade_train.index[4]

('CE103', 4)